In [1]:
import sys
import os
from os.path import abspath
import numpy as np
import pandas as pd
from utils.generate_network import generate_network
from utils.prepare_data import prepare_data
from utils.popphy_io import save_params, load_params
from utils.popphy_io import get_stat, get_stat_dict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from models.PopPhy import PopPhyCNN
import warnings
from datetime import datetime
import json
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
#from models.PopPhy2 import ResNet
from models.PopPhy2 import ResNet

# Data Preparation

### Reading Configuration
Configuring which data to read in, minimun threshold needed in an OTU (individual sample must have at least set threshold relative abundance), and how many k folds for k fold cross validation.

In [2]:
dataset = 'skg2-wt-t14'
threshold = 0
k = 5

### Reduce Features
Reduce amount of OTU features by filtering out OTUs that contain no individual sample with a relative abundance greater than the set threshold.

In [3]:
path = "../data/" + dataset
data = pd.read_csv(path + '/abundance.tsv', index_col=0, sep='\t', header=None)
to_drop = data.loc[(data < threshold).all(axis=1)]
data = data.drop(to_drop.index)

data

,1,2,3
0,,,
k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides|s__Bacteroides_acidifaciens,2071,237,1189
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_murinus,150,327,435
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_johnsonii,3,0,0
k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_unclassified,268,900,427
k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides|s__Bacteroides_vulgatus,1109,192,264
...,...,...,...
k__Bacteria|p__Firmicutes|c__Clostridia|o__Lachnospirales|f__Lachnospiraceae|g__28-4|s__28-4_unclassified,0,13,0
k__Bacteria|p__Firmicutes|c__Clostridia|o__Oscillospirales|f__Butyricicoccaceae|g__Butyricicoccus|s__Butyricicoccus_unclassified,0,13,0
k__Bacteria|p__Firmicutes|c__Clostridia|o__Oscillospirales|f__Ruminococcaceae|g__Incertae Sedis|s__Incertae Sedis_unclassified,0,12,0


### Create 2d Matrix Representing OTU Data
Dai et al. PopPhy-CNN's (2019) algorithm creates Phylogenetic tree from OTUs and populates tree based on OTU abundances. This tree graph structure is then converted to a 2d Matrix by taking each parent node in the tree graph and pushing them all to the left and childrens' nodes in the same order from left to right the parents were ordered.

In [4]:
my_maps, raw_x, tree_x, raw_features, tree_features, labels, label_set, g, feature_df = prepare_data(path, data)

# norms = np.linalg.norm(my_maps, axis=2, keepdims=True)
# my_maps = my_maps / norms
pd.DataFrame(my_maps[0])

(297, 3)
There are 297 raw features...
Building tree structure...
Tree file not found...
Constructing tree..
Pruning Tree...
Populating trees...
There are 199 tree features...


,0,1,2,3,4,5,6,7,8,9,...,262,263,264,265,266,267,268,269,270,271
0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.408199,0.372348,0.019441,0.000629,0.198925,0.000457,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.047630,0.000572,0.359998,0.372348,0.019441,0.000629,0.196180,0.002516,0.000000,0.000229,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.010349,0.037281,0.000572,0.359998,0.000000,0.372348,0.019441,0.000629,0.185145,0.000229,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.010349,0.037281,0.000572,0.359998,0.000000,0.000000,0.372348,0.019441,0.000629,0.181543,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,0.010349,0.037281,0.000572,0.359998,0.000000,0.000000,0.372348,0.019441,0.000629,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,0.005832,0.004517,0.000000,0.037281,0.000000,0.000000,0.000000,0.000000,0.000000,0.013723,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,0.005832,0.001887,0.000000,0.000000,0.000000,0.006461,0.013723,0.215107,0.000000,0.000000,...,0.000172,0.002058,0.000229,0.0,0.0,0.0,0.0,0.0,0.0,0.000229
9,0.001487,0.000000,0.012236,0.002459,0.063411,0.000000,0.009606,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


### Creating training and test sets
Splitting data into k training and k test sets

In [5]:
#one hot encoding
input = my_maps
target = tf.keras.utils.to_categorical(labels, 2, dtype='int64')
    

# #shuffle dataset
# seed = np.random.randint(100)
# # np.random.seed(seed)
# # np.random.shuffle(input)
# np.random.seed(seed)
# np.random.shuffle(target)

# np.random.seed(seed)
# np.random.shuffle(my_maps)
# np.random.seed(seed)
# np.random.shuffle(raw_x)
# np.random.seed(seed)
# np.random.shuffle(tree_x)
# np.random.seed(seed)
# np.random.shuffle(labels)


#create k training and k test sets
# groups_input = []
# groups_target = []
# k_size = len(input)//k
# start, end = 0, k_size
# for i in range(k):
#     if i == k-1:
#         group_input = input[start:]
#         group_target = target[start:]
#     else:
#         group_input = input[start:end]
#         group_target = target[start:end]
#     start += k_size
#     end += k_size
#     groups_input.append(group_input)
#     groups_target.append(group_target)

# x_train = []
# y_train = []
# x_test = []
# y_test = []
# for i in range(k-1, -1, -1):
#     x_train.append(np.concatenate((groups_input[i-1], groups_input[i-2], groups_input[i-3], groups_input[i-4])))
#     y_train.append(np.concatenate((groups_target[i-1], groups_target[i-2], groups_target[i-3], groups_target[i-4])))

#     x_test.append(groups_input[i])
#     y_test.append(groups_target[i])

# Model

### Training model
Data is log transformed and then a MinMax transformation. Uses CNN that employs skipped residual identity blocks borrowed from the classic ResNet model then a FC Neural Network to make phenotype prediction. Model dimensions printed below.

In [6]:
data_lst = []

# for i in range(k):
#     x_train1 = x_train[i]
#     y_train1 = y_train[i]
#     x_test1 = x_test[i]
#     y_test1 = y_test[i]

#     model = ResNet(height = x_train1.shape[1], width = x_train1.shape[2], channels = 1, classes = 2)
#     model.init_model()

#     model.train(x_train1, y_train1, x_test1, y_test1, dataset, use_weights = False)
#     y_pred = model.predict(x_test1)
#     auc_roc, auc_pr, f1, mcc = model.evaluate(y_test1, y_pred)
#     data_lst.append([auc_roc, auc_pr, f1, mcc])
    
#     #model.model.save_weights(path + "/model_weights.h5")

#     print(y_test1)
#     print(y_pred)
    
# print(model.model.summary())



# n_values = np.max(labels) + 1
# labels_oh = np.eye(n_values)[labels]
# tree_row = my_maps.shape[1]
# tree_col = my_maps.shape[2]

# skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
# fold = 0
# for train_index, test_index in skf.split(my_maps, labels):
#     train_x, test_x = my_maps[train_index,:,:], my_maps[test_index,:,:]
#     train_y, test_y = labels_oh[train_index,:], labels_oh[test_index,:]
        
#     train_x = np.log(train_x + 1)
#     test_x = np.log(test_x + 1)
        
#     c_prob = [0] * len(np.unique(labels))
#     train_weights = []

#     for l in np.unique(labels):
#         a = float(len(labels))
#         b = 2.0 * float((np.sum(labels==l)))
#         c_prob[int(l)] = a/b

#     c_prob = np.array(c_prob).reshape(-1)

#     for l in np.argmax(train_y, 1):
#         train_weights.append(c_prob[int(l)])
#     train_weights = np.array(train_weights)
        
#     scaler = MinMaxScaler().fit(train_x.reshape(-1, tree_row * tree_col))
#     train_x = np.clip(scaler.transform(train_x.reshape(-1, tree_row * tree_col)), 0, 1).reshape(-1, tree_row, tree_col)
#     test_x = np.clip(scaler.transform(test_x.reshape(-1, tree_row * tree_col)), 0, 1).reshape(-1, tree_row, tree_col)

#     train = [train_x, train_y]
#     test = [test_x, test_y]

#     x_train1 = train_x
#     y_train1 = train_y
#     x_test1 = test_x
#     y_test1 = test_y
        
#         y_train1 = train_y
#         y_test1 = test_y
        
#         x_train1 = np.zeros(train_x.shape)
#         x_train1[train_x != 0] = 1
        
#         x_test1 = np.zeros(test_x.shape)
#         x_test1[test_x != 0] = 1
        
        # for i in range(len(train_x)):
        #     for j in range(len(test_x)):
        #         if np.array_equal(train_x[i], test_x[j]):
        #             print('train')
        #             print(train_x[i])
        #             print('test')
        #             print(test_x[j])
        
        
#     model = ResNet(height = train_x.shape[1], width = train_x.shape[2], channels = 1, classes = 2)
#     model.init_model()
#     model.train(train_x, train_y, test_x, y_test1, dataset, use_weights = False)
#     y_pred = model.predict(test_x)
#     auc_roc, auc_pr, f1, mcc = model.evaluate(test_y, y_pred)
#     data_lst.append([auc_roc, auc_pr, f1, mcc])
#     #model.model.save_weights(path + "/model_weights.h5")
#     print(test_y)
#     print(y_pred)
#     print(model.model.summary())
    
#     fold += 1
#run += 1

model = ResNet(height = input.shape[1], width = input.shape[2], channels = 1, classes = 2)
model.init_model()
model.train(input, target, input, target, dataset, use_weights = False)
y_pred = model.predict(input)
auc_roc, auc_pr, f1, mcc = model.evaluate(target, y_pred)
data_lst.append([auc_roc, auc_pr, f1, mcc])
#model.model.save_weights(path + "/model_weights.h5")
print(target)
print(y_pred)
print(model.model.summary())
            
        
        

Metal device set to: Apple M1 Pro
Epoch 1/150
1/1 [==============================] - 2s 2s/step - loss: 3.1814 - accuracy: 0.0000e+00 - val_loss: 3.0385 - val_accuracy: 0.0000e+00
Epoch 2/150
1/1 [==============================] - 0s 211ms/step - loss: 2.8313 - accuracy: 1.0000 - val_loss: 2.8927 - val_accuracy: 1.0000
Epoch 3/150
1/1 [==============================] - 0s 177ms/step - loss: 2.6251 - accuracy: 1.0000 - val_loss: 2.7516 - val_accuracy: 1.0000
Epoch 4/150
1/1 [==============================] - 0s 186ms/step - loss: 2.4721 - accuracy: 1.0000 - val_loss: 2.6162 - val_accuracy: 1.0000
Epoch 5/150
1/1 [==============================] - 0s 227ms/step - loss: 2.3337 - accuracy: 1.0000 - val_loss: 2.4873 - val_accuracy: 1.0000
Epoch 6/150
1/1 [==============================] - 0s 149ms/step - loss: 2.2013 - accuracy: 1.0000 - val_loss: 2.3650 - val_accuracy: 1.0000
Epoch 7/150
1/1 [==============================] - 0s 150ms/step - loss: 2.0797 - accuracy: 1.0000 - val_loss: 2.24

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

# Displaying Accuracy Metrics and Saving Metrics

Option to save results of all k folds and weights of last model into same directy as data.

In [ ]:
col = [str(i) for i in range(1,k+1)]    
results_df = pd.DataFrame(data_lst, columns = ['auc(roc)', 'auc(pr)', 'f1', 'mcc'])
results_df = results_df.transpose()
results_df.columns = col

#results_df.to_csv(path + "/results.csv")
results_df

# Saving Model Weights

Option to save model weights of last model in k fold into same directy as data.

In [ ]:
#model.model.save_weights(path + "/model_weights.h5")

## 